In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#reading train and test dataset for customer prediction
df_train = pd.read_csv("/kaggle/input/ga-customer-revenue-prediction/train.csv")

#reading dataset
#df_test = pd.read_csv("/kaggle/input/ga-customer-revenue-prediction/test.csv")

In [ ]:
df_train.head()

In [ ]:
#convert json columns

import os
import json
from pandas.io.json import json_normalize

def load_df(csv_path='/kaggle/input/ga-customer-revenue-prediction/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
train_df = load_df('/kaggle/input/ga-customer-revenue-prediction/train.csv')


In [ ]:
#test_df = load_df('/kaggle/input/ga-customer-revenue-prediction/test.csv')

In [ ]:
#better description for dataset

from scipy import stats


def DataDesc(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 
    
    return summary

In [ ]:
DataDesc(train_df)

In [ ]:

def fill_na(df):   
    df['totals.pageviews'].fillna(1, inplace=True)
    df['totals.newVisits'].fillna(0, inplace=True)
    df['totals.bounces'].fillna(0, inplace=True) 
    df["totals.transactionRevenue"].fillna(0.0, inplace=True)
    
    # Changing datatypes from object to desired ones
    df['totals.pageviews'] = df['totals.pageviews'].astype(int)
    df['totals.newVisits'] = df['totals.newVisits'].astype(int)
    df['totals.bounces'] = df['totals.bounces'].astype(int)
    df["totals.transactionRevenue"] = df["totals.transactionRevenue"].astype(float)
    
    
    df['trafficSource.isTrueDirect'].fillna(False, inplace=True) 
    df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True) # filling boolean with True
    df[train_df['geoNetwork.city'] == "(not set)"]['geoNetwork.city'] = np.nan
    df['geoNetwork.city'].fillna("NaN", inplace=True)
    
    return df

df = fill_na(train_df)
#imputation of null values and converting the columns values to int


In [ ]:
#converting to float

df['fullVisitorId'] = df['fullVisitorId'].astype(float)

df['sessionId'] = df['sessionId'].astype(float)



#df1['fullVisitorId'] = df1['fullVisitorId'].astype(float)

#df1['sessionId'] = df1['sessionId'].astype(float)

In [ ]:
df.info()

In [ ]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)

In [ ]:
# This function is to extract date features

from datetime import datetime


def date_process(df):
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d") # seting the column as pandas datetime
    df["weekday"] = df['date'].dt.weekday #extracting week day
    df["day"] = df['date'].dt.day # extracting day
    df["month"] = df['date'].dt.month # extracting day
    df["year"] = df['date'].dt.year # extracting day
    df['visitHour'] = (df['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))).astype(int)
    
    return df
df = date_process(df)

In [ ]:
df = df.drop(['sessionId', 'trafficSource.campaignCode'], axis = 1)

In [ ]:
df["totals.transactionRevenue"].fillna(0, inplace=True)
train_y = df["totals.transactionRevenue"].values
train_id = df["fullVisitorId"].values

In [ ]:
from sklearn import model_selection, preprocessing, metrics

# label encode the categorical variables 

df_cat = ["channelGrouping", "device.browser", 'device.isMobile',
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']

In [ ]:
for col in df_cat:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df[col].values.astype('str')))
    df[col] = lbl.transform(list(df[col].values.astype('str')))

In [ ]:
num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    df[col] = df[col].astype(float)

In [ ]:
train_y = np.log1p(df["totals.transactionRevenue"].values)

In [ ]:
train_x = df.drop(['totals.transactionRevenue'], axis = 1)

In [ ]:
train_y = pd.DataFrame(train_y)   
train_y.info()

In [ ]:
train_x.info()

In [ ]:
X= train_x
Y= train_y

In [ ]:
X['date'].head()

In [ ]:
df["date"] = pd.to_datetime(df["date"]).dt.date
df['date']

In [ ]:
import datetime

X = df[df['date']<=datetime.date(2017,5,31)]
val_X = df[df['date']>datetime.date(2017,5,31)]
X = X.drop(['date'], axis = 1)
val_X = df.drop(['date'], axis = 1)

In [ ]:
y = np.log1p((X["totals.transactionRevenue"]).values)
val_y = np.log1p((val_X["totals.transactionRevenue"]).values)

In [ ]:
y = pd.DataFrame(y)
val_y = pd.DataFrame(val_y)

In [ ]:
x = X.drop(['totals.transactionRevenue'], axis = 1)
val_x = val_X.drop(['totals.transactionRevenue'], axis = 1)

In [ ]:
print(x.shape,val_x.shape)
print(y.shape,val_y.shape)

In [ ]:
import xgboost as xg 

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
model = xg.XGBRegressor(objective ='reg:squarederror', n_estimators = 1000, verbosity=1, learning_rate=0.3)
model.fit(x, y)

In [ ]:
model.score(x,y)